<center>
    <h1>Modeling an Electron in a Field of Secured Point Charges</h1>
</center>

<h2>Introduction</h2>

Kinetics typically makes sense. You can see kinetics. But, you can't really see an electric field. The idea that an electron will move towards a positive charge and away from a negative charge might make sense, but it can be difficult to visualize where an electron might go in a more complicated field of charges. This project is about helping students in physics classes understand how electrons are moved in electric fields

In my project, we are looking at an idealized simulation. The only forces present on the electron are the forces created by the charges, and those charges are all point charges fixed in place that won't be affected by each other. While not an extrememly realistic situation, it is a good situation for beginners to get a better understanding.

The simulation relies on Coulomb's Law, which is used to calculate the force on a point charge by another point charge. It also uses Newton's Second Law (f=ma) to get acceleration from force. Finally, we use Reimann sums, as velocity is the area under the curve of acceleration and position the area under the curve of velocity.

<h2>Methodology</h2>

<h3>Imports and Libraries</h3>

In [1]:
import math #basic python math library, used for square rooting and getting pi
#https://docs.python.org/3/library/math.html#
import numpy as np #common library, source of numpy arrays which are easier to manipulate as vectors than your average python list
#https://numpy.org/doc/stable/index.html

from scipy.constants import e, m_e, epsilon_0 
#physics constants I need, the charge of a proton or electron (e), the mass of an electron (m_e), and epsilon naught
#https://docs.scipy.org/doc/scipy/reference/constants.html

import matplotlib.pyplot as plt #for plotting data
#https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
import matplotlib.animation as animation #to create animation
#https://matplotlib.org/stable/api/animation_api.html
from IPython.display import HTML #making it possible to display my animation
#https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

<h3>Basic Objects (Charge)</h3>

In [2]:
class Charge: #class to create Charge objects, which will make it easier to work with charges

    def __init__(self, x, y, charge): #initiation function, magic method
        self.position = np.array([float(x), float(y)]) ##creating the position, using the given x and y coordinates, as an array
        #https://numpy.org/doc/stable/reference/generated/numpy.array.html
        self.charge = charge #getting the charge of the particle

    def get_position(self): #function to get the position of a charge
        return self.position #return the position

    def get_charge(self): #function to get the charge of a charge
        return self.charge #return the charge

<h3>Complicated Objects (Electron)</h3>

In [3]:
class Electron: #class to create Electron objects, which will allow me to make functions for electrons

    def __init__(self, x, y): #initiation function, magic method
        self.position = np.array([float(x), float(y)]) #creating the position, using the given x and y coordinates, as an array
        self.charge = -e #all electrons have a charge of -e
        self.velocity = np.array([0.0, 0.0]) #start with 0 velocity
        self.acceleration = np.array([0.0, 0.0]) #start with 0 acceleration

    def get_position(self): #function to get the position of an electrno
        return self.position #return the position

    def get_velocity(self):
        return self.velocity
    
    def get_acceleration(self): #return the acceleration
        return self.acceleration #return acceleration

    def get_force(self, charges): #function to get the sum of all the forces on an electron
        force = np.array([0.0, 0.0]) #start with 0
        for charge in charges: #then go through each charge
            r = math.sqrt((charge.get_position() - self.get_position())[0] ** 2 + (charge.get_position() - self.get_position())[1] ** 2)
            #get the distance from electron to charge
            force += (self.get_position() - charge.get_position()) * charge.get_charge() * self.charge / 4 / math.pi / epsilon_0 / (r ** 3)
            #use Coulomb's Law to find the force a single charge places on the electron and add it to the current force
        self.acceleration = force / m_e / 100 #Newton's second law, f=ma, so a=f/m

    def update_velocity(self): #changing velocity
        self.velocity += self.acceleration #small increments, velocity changes by acceleration

    def update_position(self): #changing position
        self.position += self.velocity #small increments, position changes by velocity

<h3>Repeated Function</h3>

In [4]:
def electron_update(charges, electron): #function to update the state of the electron

    electron.get_force(charges) #getting the current force
    electron.update_velocity() #updating velocity
    electron.update_position() #updating position

<h3>Generator Function</h3>

In [5]:
def gen(charges, electron): #making a generator function!
    i = 1 #start with one frame
    sep = True #boolean to determine if you should continue
    while sep:
        #print(i)
        for charge in charges: #look at each charge
            if abs(electron.get_position()[0] - charge.get_position()[0]) < 2 and abs(electron.get_position()[1] - charge.get_position()[1]) < 2:
            #if the electron is practically on the charge
                sep = False #don't continue
                print("on charge") #let us know we are on a charge
                print(charge.get_position(), electron.get_position()) #print both locations
                print(abs(electron.get_position()[0] - charge.get_position()[0]), abs(electron.get_position()[1] - charge.get_position()[1])) #print distance
                break #exit this loop
        if abs(electron.get_position()[0]) > 100 or abs(electron.get_position()[1]) > 100: #if it's outside of the boundaries of the simulation
            print("left boundaries")
            sep = False #don't continue
        if np.array_equal(electron.get_velocity(), [0.0,0.0]) and np.array_equal(electron.get_acceleration(), [0.0, 0.0]): #if the velocity + acceleration is 0
            print("no velocity/acceleration") #let us know that the velocity/acceleration is 0
            print(electron.get_velocity(), electron.get_acceleration()) #prove it
            sep = False #don't continue
        if sep == True: #if we still should continue
            i += 1 #up the frame number
            #print(i)
            yield i #iteration version of return

<h3>Modeling</h3>

In [6]:
def create_animation(charges, electron): #creating the animation

    fig, ax = plt.subplots() #needed elements of the plot
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
    
    scatter = ax.scatter([c.get_position()[0] for c in charges if c.get_charge() > 0], [c.get_position()[1] for c in charges if c.get_charge() > 0], c="r", label="+ charges")
    #placing all of the positive charges on the plot, coloring them red
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.scatter.html#matplotlib.axes.Axes.scatter
    scatter = ax.scatter([c.get_position()[0] for c in charges if c.get_charge() < 0], [c.get_position()[1] for c in charges if c.get_charge() < 0 ], c="b", label="- charges")
    #placing all of the negative charges on the plot, coloring them blue
    scatter = ax.scatter(electron.get_position()[0], electron.get_position()[1], c="g", label="electron")
    #place the electron on the plot, coloring it green

    ax.set_xlim([-100, 100]) #making the x-axis -100 to 100
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_xlim.html
    ax.set_ylim([-100, 100]) #making the y-axis -100 to 100
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_ylim.html
    ax.legend() #show the legend
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.legend.html
    
    def update(frame): #update function!

        electron_update(charges, electron) #call the update function to set the values
        scatter.set_offsets(electron.get_position()) #update the position of the electron
        #https://matplotlib.org/stable/api/collections_api.html#matplotlib.collections.AsteriskPolygonCollection.set_offsets
        return scatter #return the new scatter plot

    electron.get_force(charges) #not have starting acceleration 0
    anim = animation.FuncAnimation(fig=fig, func=update, frames=gen(charges, electron), interval=10, save_count=1000) #create the animation
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html
    plt.close() #close the plot

    return HTML(anim.to_jshtml()) #return the HTML to make the animation
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.Animation.html#matplotlib.animation.Animation.to_jshtml

<h3>The Program</h3>

In [ ]:
print("This program will create a 2d simulation with point charges fixed in place and an unsecured electron.")
print("The dimensions are (-100, 100) by (-100, 100).")
print()

print("First, create your Charges.")
q = int(input("Number of Charges (int): "))
charges = []
for i in range(q):
    x = float(input("Enter x coordinate (float): "))
    y = float(input("Enter y coordinate (float): "))
    c = float(input("Enter charge (float): "))
    charges.append(Charge(x, y, c * e))
    
print("Second, locate your Electron.")
x = float(input("Enter x coordinate (float): "))
y = float(input("Enter y coordinate (float): "))
electron = Electron(x, y)

create_animation(charges, electron)

<h2>Results and Discussion</h2>

<h2>Conclusion</h2>

<h2>Citations</h2>

<p>
    ImportanceOfBeingErnest. (2018, February 1). <em>How to stop FuncAnimation by func
     in matplotlib? </em>[Online forum post]. Stack Overflow.
     https://stackoverflow.com/questions/48564181/
     how-to-stop-funcanimation-by-func-in-matplotlib
</p>
<p>
    <em>Matplotlib.animation.FuncAnimation</em>. (n.d.). Matplotlib. Retrieved May 9, 2024,
     from https://matplotlib.org/stable/api/_as_gen/
     matplotlib.animation.FuncAnimation.html
</p>
<p>
    <em>Matplotlib.animation.FuncAnimation</em>. (n.d.). Matplotlib. Retrieved May 9, 2024,
     from https://matplotlib.org/stable/api/_as_gen/
     matplotlib.animation.FuncAnimation.html
</p>
<p>
    <em>The n-dimensional array (ndarray)</em>. (n.d.). NumPy. Retrieved May 9, 2024, from
     https://numpy.org/doc/stable/reference/arrays.ndarray.html
</p>